In [1]:
from typing import Tuple
import numpy as np


def sine(amplitude: float, frequency: float, time: float, phase: float) -> np.ndarray:
    return amplitude * np.sin(2 * np.pi * frequency * time + phase)


def tone(sample_rate: float, time_of_view: float, **kwargs: float) -> Tuple[np.ndarray, ...]:
    sample_period = 1 / sample_rate
    total_samples = time_of_view / sample_period
    time_points = np.linspace(0, time_of_view, int(total_samples))
    return time_points, sine(**kwargs, time=time_points)


def play_tone(signal: np.ndarray, play: bool) -> None:
    if play:
        sound_wave = signal.astype(np.int16)
        sd.stop()
        sd.play(sound_wave, blocking=False)
    else:
        sd.stop()


In [6]:
import sounddevice as sd

# TODO: find the proper frequencies for each tone
# Configure tone values
sampling_rate = 44_100
amplitude = 10_000
time_of_view = 2
phase = 0

# Configure the player
sd.default.samplerate = sampling_rate

# Map tone values to frequencies
tone_frequencies = [
    ('Do', 440),
    ('Re', 493.883),
    ('Mi', 554.3653),
    ('Fa', 587.3295),
    ('Sol', 659.2551),
    ('La', 739.9888),
    ('Si', 830.6094),
    ('Do\'', 880)
]

# Create tone values
time_points = None
tones = dict()

# Instantiate each tone
for (note, freq) in tone_frequencies:
    time_points, tones[note] = tone(
        amplitude=amplitude,
        frequency=freq,
        sample_rate=sampling_rate,
        time_of_view=time_of_view,
        phase=phase
    )


In [10]:
def play_sound(note, play):
    play_tone(tones[note], play)

In [11]:
from IPython.display import display
import ipywidgets as widgets

play_toggle = widgets.ToggleButton(
    value=False,
    description='Play Tone',
    disabled=False,
    button_style='',
    tooltip='Play Tone',
    icon='play'
)

dropdown = widgets.Dropdown(
    options=[note for (note, freq) in tone_frequencies],
    value=tone_frequencies[0][0],
    description='Tone'
)

controls = widgets.interactive(play_sound, note=dropdown, play=play_toggle)

h_box = widgets.HBox(
    controls.children
)

display(h_box)